<a href="https://colab.research.google.com/github/Rumaisa1054/Data_Science/blob/main/time_series/time_series_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# preprocessing
1. sort
2. handle missing data or abnormal data